## LLama Model practicing the Promt enginnering
-  LLaMA2
- Chatbots: Create intelligent chatbots that can handle complex conversations.

In [3]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.82-cp310-cp310-linux_x86_64.whl size=2815281 sha256=0434ad47ecb6dd20655dd0bd6a526f0bf46f1e3cc76fcc4bb4114e52ab61e94c
  Stored in directory: /root/.cache/pip/wheels/5f/da/5a/272c969ba31c678e6bee2543f3be4dfa67d0feb36ed6f94c01
Successfully built llama-cpp-python


## Install the llama Cpp Python model in the system
- Load a LLAMA model from the Hugging Face Model Hub using transformers and integrate it with llama-cpp-python.

In [1]:
pip install huggingface_hub

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Llama
- what are the steps ?
- Take The Model_name_path  of the model
- Take base_model
- Pass the parameter in the hf_hub_download


In [5]:
Model_name_path ="TheBloke/Llama-2-7B-Chat-GGUF"
base_model="llama-2-7b-chat.Q8_0.gguf"

In [6]:
# use the hf_hub_download  to download the modell
model_path= hf_hub_download(repo_id=Model_name_path,filename=base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-7b-chat.Q8_0.gguf:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

In [7]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [8]:
lccp_llm=Llama(model_path=model_path,
               n_batch=512,
               n_gpu_layers=32
               ,n_ctx=4096,
               n_threads=2)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_coun

In [9]:
lccp_llm

In [10]:
# now first step system_message Need to create the system_message
system_message = """You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user"""

In [11]:
# create the answer template how the answer need to be given to the user
answer_template = """
Context:
{context}
===
Using the context above generated {num_answer} distinct answer to the following quary Question
{qustion}.
Arrange your answer in thew bullet point.
present answer in the bullet points."""

In [12]:
AXA_2000_context ="""
As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis breaks out in summer 2007, AXA has learned the lessons from 2001. Thanks to its efficient internal risk management system (GRM), the Group is affected, but shows its resilience, without public bailouts.

In this volatile environment, AXA remains convinced of the importance of its long-term commitments to its customers and society in general. In line with this spirit, AXA launches the AXA Research Fund to improve the understanding and prevention of major risks around the world. The AXA Research Fund further enhances the wide range of initiatives in place across the Group, which is committed to taking action on what counts for communities. Created out of the belief that science has a key role to play in meeting society’s major challenges, AXA sets out its commitment to help drive collective progress by supporting research projects focused on health, the environment, new technologies and socioeconomics.
"""

In [13]:
actual_question = "When the subprime financial crisis breaks out? What the company had learned?"

## Base Template
- llama_prompt_template has the base template That is written in the certain pattern
- `Ins` is the instruction
- `<<sys>>` is start of the system message
- `{system}` is the template driven where we are passing the certain instructions
- `{user_message}` It is template driven where it is passed certain Instruction

In [14]:
llama_prompt_template='''[/s][INS]<<SYS>>
{system_message}
<</SYS>>
{user_message}[/INST]'''

## Self-consistency
- In the Self-Consistency the we generate number of answer(num_answer=3)  for a single question. with this the model will pick most occurance answer across the occurrences.

In [15]:
answer_prompt=llama_prompt_template.format(
    system_message=system_message,
    user_message=answer_template.format(
        context=AXA_2000_context,
        qustion=actual_question,
        num_answer=3
    )
)

In [16]:
answer_prompt

'[/s][INS]<<SYS>>\nYou are an assistance you are answer a quary on a finacial information\nDonot reapeat the question only answer the question presented by the user\n<</SYS>>\n\nContext:\n\nAs soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).\n\nThe Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).\n\nIn 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.\n\nWhen the subprime financial crisis bre

In [17]:
print(answer_prompt)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

Context:

As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis breaks out in sum

In [18]:
answer_promt = llama_prompt_template.format(
    system_message=system_message,
    user_message=answer_template.format(
      context=AXA_2000_context,
      num_answer=4,
      qustion=actual_question
    )
)

In [19]:
answer_promt

'[/s][INS]<<SYS>>\nYou are an assistance you are answer a quary on a finacial information\nDonot reapeat the question only answer the question presented by the user\n<</SYS>>\n\nContext:\n\nAs soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).\n\nThe Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).\n\nIn 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.\n\nWhen the subprime financial crisis bre

In [20]:
print(answer_promt)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

Context:

As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis breaks out in sum

In [21]:
response=lccp_llm(prompt=answer_promt,max_tokens=1500,temperature=0.2,top_p=0.2,min_p=0.05
)


llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =     114.21 ms /   180 runs   (    0.63 ms per token,  1576.10 tokens per second)
llama_print_timings: prompt eval time =   90406.35 ms /   509 tokens (  177.62 ms per token,     5.63 tokens per second)
llama_print_timings:        eval time =   75750.57 ms /   179 runs   (  423.19 ms per token,     2.36 tokens per second)
llama_print_timings:       total time =  166475.77 ms /   688 tokens


In [22]:
print(response)

{'id': 'cmpl-41588c81-e88c-4709-b51a-4f9e353651c8', 'object': 'text_completion', 'created': 1721232086, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q8_0.gguf', 'choices': [{'text': '  When the subprime financial crisis broke out in summer 2007, AXA had learned several important lessons from its previous experience in 2001:\n• Efficient internal risk management system (GRM): AXA had developed a robust risk management system, which helped it to weather the crisis without public bailouts.\n• Resilience: Despite being affected by the crisis, AXA demonstrated its resilience and ability to adapt to challenging situations.\n• Long-term commitments: AXA remained committed to its long-term commitments to its customers and society in general, even during times of economic uncertainty.\n• Importance of research: AXA recognized the importance of investing in research to improve understanding and pr

In [23]:
factual_answer= response["choices"][0]['text']
print(factual_answer)

  When the subprime financial crisis broke out in summer 2007, AXA had learned several important lessons from its previous experience in 2001:
• Efficient internal risk management system (GRM): AXA had developed a robust risk management system, which helped it to weather the crisis without public bailouts.
• Resilience: Despite being affected by the crisis, AXA demonstrated its resilience and ability to adapt to challenging situations.
• Long-term commitments: AXA remained committed to its long-term commitments to its customers and society in general, even during times of economic uncertainty.
• Importance of research: AXA recognized the importance of investing in research to improve understanding and prevention of major risks around the world, launching the AXA Research Fund.


In [24]:
consistancy_template = """
Here are {num_answer} answer for the question mention below:
Question:
{question}
Answer:
{Answer}
Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:
"""

In [25]:
consistancy_prompt= llama_prompt_template.format(
    system_message=system_message,
    user_message=consistancy_template.format(
        num_answer=4,
        question=actual_question,
        Answer=factual_answer
    )
)

In [26]:
response= lccp_llm(prompt=consistancy_prompt,temperature=0.2,echo=False,top_p=0.80,min_p=0.09,max_tokens=1500)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      44.84 ms /    70 runs   (    0.64 ms per token,  1561.04 tokens per second)
llama_print_timings: prompt eval time =   45523.46 ms /   250 tokens (  182.09 ms per token,     5.49 tokens per second)
llama_print_timings:        eval time =   28128.24 ms /    69 runs   (  407.66 ms per token,     2.45 tokens per second)
llama_print_timings:       total time =   73804.10 ms /   319 tokens


In [27]:
response

{'id': 'cmpl-1d8e4fcb-cb2f-4a91-b437-cc6b78d6c7c4',
 'object': 'text_completion',
 'created': 1721232252,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q8_0.gguf',
 'choices': [{'text': '  Final Answer: When the subprime financial crisis broke out in summer 2007, AXA had learned several important lessons from its previous experience in 2001, including the importance of efficient internal risk management system (GRM), resilience, long-term commitments, and the importance of research.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 303, 'completion_tokens': 69, 'total_tokens': 372}}

### Final Response with Self-consistency

In [28]:
print(response )

{'id': 'cmpl-1d8e4fcb-cb2f-4a91-b437-cc6b78d6c7c4', 'object': 'text_completion', 'created': 1721232252, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q8_0.gguf', 'choices': [{'text': '  Final Answer: When the subprime financial crisis broke out in summer 2007, AXA had learned several important lessons from its previous experience in 2001, including the importance of efficient internal risk management system (GRM), resilience, long-term commitments, and the importance of research.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 303, 'completion_tokens': 69, 'total_tokens': 372}}


In [29]:
print(response["choices"][0]["text"])

  Final Answer: When the subprime financial crisis broke out in summer 2007, AXA had learned several important lessons from its previous experience in 2001, including the importance of efficient internal risk management system (GRM), resilience, long-term commitments, and the importance of research.


## Tree-of-Thought
- In the Tree of the facts the we will generate answers n_number=3 and with the percentage/ rank score will give the best answer

In [30]:
solution_template ="""
Generate {number_answer} answer for the following question

problem:
{problem}
consider the following factor compute the following answer
factor:
{factors}
Present the solutions in numbered bullet points. Present only the solutions.
"""


In [31]:
factors="""
1.Risk appetite
2.Risk culture
3.Risk governance
4.Compliance
5.Risk mitigation
6.Risk identification
"""


In [32]:
tree_of_thoughts= llama_prompt_template.format(
    system_message=system_message,
    user_message=solution_template.format(
        number_answer=3,
        problem=AXA_2000_context,
        factors=factors,
    )
)

In [33]:
response = lccp_llm(prompt=tree_of_thoughts,echo=False,max_tokens=1500,frequency_penalty=0.2,top_p=.8)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =     289.00 ms /   379 runs   (    0.76 ms per token,  1311.44 tokens per second)
llama_print_timings: prompt eval time =   92101.58 ms /   488 tokens (  188.73 ms per token,     5.30 tokens per second)
llama_print_timings:        eval time =  188204.49 ms /   378 runs   (  497.90 ms per token,     2.01 tokens per second)
llama_print_timings:       total time =  281252.73 ms /   866 tokens


In [34]:
print(response["choices"][0]["text"])

  Sure, here are the solutions for the factors mentioned:
1. Risk appetite:
* AXA should establish a clear and well-defined risk appetite statement that reflects its tolerance for risk and guides decision-making across the organization.
2. Risk culture:
* AXA should foster a strong risk culture by promoting open communication, transparency, and accountability at all levels of the organization. This can be achieved through regular training and awareness programs, as well as incentivizing employees to speak up when they identify potential risks.
3. Risk governance:
* AXA should establish a robust risk governance framework that clearly defines roles and responsibilities for risk management across the organization. This should include the appointment of a Chief Risk Officer (CRO) who reports directly to the CEO and oversees the Group's risk management activities.
4. Compliance:
* AXA should maintain a comprehensive compliance program that ensures adherence to relevant laws, regulations, an

In [35]:
solution_response=response["choices"][0]["text"]

In [36]:
evaluation_template = """
For the following problem: {problem}, evaluate each solution in the following proposed solutions: \n{solutions}\n.

Analyze pros, cons, feasibility, and probability of success for each solution.
Present your evaluations of each solutions.
"""

In [37]:
evalution_prompt= llama_prompt_template.format(
    system_message=system_message,
    user_message=evaluation_template.format(
        problem=AXA_2000_context,
        solutions=solution_response,
    )
)

In [38]:
print(evalution_prompt)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

For the following problem: 
As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis

In [39]:
response= lccp_llm(prompt=evalution_prompt,temperature=0.5,top_p=0.8,max_tokens=1500,echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =     438.59 ms /   669 runs   (    0.66 ms per token,  1525.35 tokens per second)
llama_print_timings: prompt eval time =  156416.26 ms /   830 tokens (  188.45 ms per token,     5.31 tokens per second)
llama_print_timings:        eval time =  333185.43 ms /   668 runs   (  498.78 ms per token,     2.00 tokens per second)
llama_print_timings:       total time =  491602.88 ms /  1498 tokens


In [40]:
print(response["choices"][0]["text"])

  Sure, here are my evaluations of the proposed solutions for AXA:
1. Risk appetite: Pros - Establishing a clear risk appetite statement can help guide decision-making and ensure that the organization is taking on appropriate levels of risk. Cons - This solution may not be feasible or effective if the organization does not have a well-defined risk management framework in place. Feasibility - 6/10 Probability of success - 7/10
2. Risk culture: Pros - Fostering a strong risk culture can help promote open communication, transparency, and accountability across the organization. Cons - This solution may be difficult to implement if the organization does not have a strong commitment to risk management. Feasibility - 8/10 Probability of success - 9/10
3. Risk governance: Pros - Establishing a robust risk governance framework can help ensure that risk management is integrated into all aspects of the organization. Cons - This solution may be resource-intensive and require significant changes to

In [41]:
ranking_template = """
For the following problem: {problem}, rank the solutions presented in the following evaluations: \n{evaluations}\n.
Pick most promising solution and present implementation strategies and methods to handle potential obstacles for this solution.
"""

In [42]:
response= lccp_llm(prompt=ranking_template,echo=False,temperature=0.5,top_p=.80,frequency_penalty=0,max_tokens=1500)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =     114.96 ms /   196 runs   (    0.59 ms per token,  1704.93 tokens per second)
llama_print_timings: prompt eval time =    8928.47 ms /    52 tokens (  171.70 ms per token,     5.82 tokens per second)
llama_print_timings:        eval time =   80057.82 ms /   195 runs   (  410.55 ms per token,     2.44 tokens per second)
llama_print_timings:       total time =   89318.55 ms /   247 tokens


### Final Response with Tree-of-thought

In [43]:
final_tree_reaponse=(response["choices"][0]["text"])
final_tree_reaponse

'\nSolution 1: {solution 1}\nStrengths:\nWeaknesses:\nOpportunities:\nThreats:\nImplementation Strategies:\nPotential Obstacles:\n\nSolution 2: {solution 2}\nStrengths:\nWeaknesses:\nOpportunities:\nThreats:\nImplementation Strategies:\nPotential Obstacles:\n\nSolution 3: {solution 3}\nStrengths:\nWeaknesses:\nOpportunities:\nThreats:\nImplementation Strategies:\nPotential Obstacles:\n\nPlease provide your evaluation of the solutions and rank them based on their feasibility, practicality, and potential impact. Also, please present any additional information or insights that may be relevant to the decision-making process.'

## Rephrase & Respond
- When generating a response, the model can create multiple possible outputs, each considered a "choice." The most suitable choice is then selected to be presented to the user.

In [44]:
rephase_respond_template="""
context:
{context}
===
Question:
{question}


Observe the context presented above, rephrase and expand the above question to help you do better answering.
Maintain all the information in the original question.
Please note that you only have to rephrase the question, do not mention the context.
The context is only presented for your reference.

Present your answer in the following format:
Rephrased Question:
<rephrased-question-here>
"""

### when do Rephrase & Respond is use full ?
- When the initial response might be confusing or not as clear as it could be, rephrasing helps to ensure the message is easily understood.
- When the response needs to be more formal or polished, rephrasing can adjust the tone and structure accordingly.
- When the original response is too complex or technical, rephrasing can make it more accessible to a broader audience.

In [45]:
rephase_response_prompt= llama_prompt_template.format(
    system_message=system_message,
    user_message= rephase_respond_template.format(
        context=AXA_2000_context,
        question=actual_question
    )
)

In [46]:
rephase_response_prompt

'[/s][INS]<<SYS>>\nYou are an assistance you are answer a quary on a finacial information\nDonot reapeat the question only answer the question presented by the user\n<</SYS>>\n\ncontext:\n\nAs soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).\n\nThe Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).\n\nIn 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.\n\nWhen the subprime financial crisis bre

In [47]:
instial_response= lccp_llm(prompt=rephase_response_prompt,temperature=0.5,top_p=0.8,max_tokens=1500,echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      28.43 ms /    46 runs   (    0.62 ms per token,  1618.12 tokens per second)
llama_print_timings: prompt eval time =  102466.94 ms /   572 tokens (  179.14 ms per token,     5.58 tokens per second)
llama_print_timings:        eval time =   19148.84 ms /    45 runs   (  425.53 ms per token,     2.35 tokens per second)
llama_print_timings:       total time =  121684.87 ms /   617 tokens


In [48]:
rephrased_question = instial_response["choices"][0]["text"].strip()

In [49]:
print(rephrased_question)

Rephrased Question: What lessons did AXA learn from the subprime financial crisis in 2007, and how did these lessons help the company navigate the crisis without public bailouts?


In [50]:
rephrase_marker = rephrased_question.find('Rephrased Question:')

In [51]:
print(rephrased_question[rephrase_marker+19:])

 What lessons did AXA learn from the subprime financial crisis in 2007, and how did these lessons help the company navigate the crisis without public bailouts?


In [52]:
rephrased_factual_question = rephrased_question[rephrase_marker+19:]

In [53]:
response_template = """
Context:
{context}
===

Original Question:
{question}

Rephrased Question:
{rephrased_question}

Given the above context, use your answer for the rephrased question presented above to answer the original question.
Present your final answer in the following format.
Final Answer: <your-final-answer>
"""

In [54]:
response_prompt = llama_prompt_template.format(
    system_message=system_message,
    user_message=response_template.format(
        context=AXA_2000_context,
        question=actual_question,
        rephrased_question=rephrased_factual_question
    )
)

In [55]:
print(response_prompt)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

Context:

As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis breaks out in sum

In [56]:
response_llama=lccp_llm(prompt=response_prompt,temperature=0.5,top_p=0.80,max_tokens=1500,echo=False)


Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      81.35 ms /   118 runs   (    0.69 ms per token,  1450.47 tokens per second)
llama_print_timings: prompt eval time =   96798.60 ms /   522 tokens (  185.44 ms per token,     5.39 tokens per second)
llama_print_timings:        eval time =   52545.83 ms /   117 runs   (  449.11 ms per token,     2.23 tokens per second)
llama_print_timings:       total time =  149559.78 ms /   639 tokens


In [57]:
print(response_llama["choices"][0]["text"])

  Final Answer: In 2007, AXA learned the importance of efficient internal risk management systems (GRM) and resilience in the face of financial crises. These lessons were crucial in navigating the subprime financial crisis without public bailouts. By implementing a comprehensive risk management strategy, AXA was able to identify potential risks and take proactive measures to mitigate them. This allowed the company to weather the storm more effectively than in 2001, when it was not as prepared for the crisis.


## Final Responce for Rephrase & Respond

In [58]:
rephase_response_llm=response_llama["choices"][0]["text"].strip()
rephase_response_llm

'Final Answer: In 2007, AXA learned the importance of efficient internal risk management systems (GRM) and resilience in the face of financial crises. These lessons were crucial in navigating the subprime financial crisis without public bailouts. By implementing a comprehensive risk management strategy, AXA was able to identify potential risks and take proactive measures to mitigate them. This allowed the company to weather the storm more effectively than in 2001, when it was not as prepared for the crisis.'

## cove (chain of verification)
- The cove will generate the multiple question and verify the the with the original question
- It take 4 steps

### Step:1 Baseline

In [59]:
## Base line
Baseline_prompt="""
content:
{content}
Question:
{question}
based on the content generate the question """

In [60]:
actual_question_1="what year financial crisis is it 2001 or 2007?"

In [61]:
Baseline= llama_prompt_template.format(system_message=system_message,
                                       user_message=Baseline_prompt.format(
                                         content=AXA_2000_context,
                                         question= actual_question_1
                                       ))

In [62]:
print(Baseline)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

content:

As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, the economic environment is positive again. AXA returns to profitable growth, acquires the Swiss firm Winterthur and starts investing in emerging markets.

When the subprime financial crisis breaks out in sum

In [63]:
baseline_llm=lccp_llm(prompt=base_model,max_tokens=1500,temperature=0.8,top_p=0.8,echo=False)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =       1.57 ms /     3 runs   (    0.52 ms per token,  1914.49 tokens per second)
llama_print_timings: prompt eval time =    3250.48 ms /    17 tokens (  191.20 ms per token,     5.23 tokens per second)
llama_print_timings:        eval time =     780.72 ms /     2 runs   (  390.36 ms per token,     2.56 tokens per second)
llama_print_timings:       total time =    4037.62 ms /    19 tokens


In [64]:
print(baseline_llm["choices"][0]["text"])

In [65]:
baseline_response=baseline_llm["choices"][0]["text"].strip()
baseline_response

### Step:2 Verification

In [66]:
verifications_prompt_template = """
Your task is to create verification questions based on the below original question and the baseline response.
The verification questions are meant for verifying the factual acuracy in the baseline response.

Context:
{context}
===

Question:
Use the above context to answer the following question: {question}

Baseline Response:
{baseline_response}

Respond with your verification questions in a JSON format with the following headers.
```JSON
question1: <verification-question-1>
question2: <veriification-question-2>
and so on.
```
Do not provide answers to these verification questions. Respond only with the JSON.
"""

In [69]:
verification_prompt= llama_prompt_template.format(system_message=system_message,
                                                  user_message=verifications_prompt_template.format(
                                                      context=AXA_2000_context,
                                                      question=actual_question_1,
                                                      baseline_response=baseline_response,
                                                  )
                                                  )

In [70]:
print(verification_prompt)

[/s][INS]<<SYS>>
You are an assistance you are answer a quary on a finacial information
Donot reapeat the question only answer the question presented by the user
<</SYS>>

Your task is to create verification questions based on the below original question and the baseline response.
The verification questions are meant for verifying the factual acuracy in the baseline response.

Context:

As soon as he takes office, Henri de Castries chooses to sell off the US investment bank Donaldson, Lufkin & Jenrette (DLJ) and target operational excellence for the Group’s core business areas (insurance and asset management).

The Group organizes itself to weather this storm: rolling out a plan to reduce spending, turning around its technical results and further strengthening its risk management, with the creation of Group Risk Management which objective is to analyze the Group’s risk exposure and appetite with a more in-depth approach and to prepare for future new regulations (Solvency 2).

In 2006, 

In [71]:
verification_prompt_llm =lccp_llm(prompt=verification_prompt,temperature=0.8,max_tokens=1500,echo=False,top_p=0.80)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      81.96 ms /   130 runs   (    0.63 ms per token,  1586.20 tokens per second)
llama_print_timings: prompt eval time =  122295.76 ms /   614 tokens (  199.18 ms per token,     5.02 tokens per second)
llama_print_timings:        eval time =   55978.72 ms /   129 runs   (  433.94 ms per token,     2.30 tokens per second)
llama_print_timings:       total time =  178491.58 ms /   743 tokens


In [72]:
print(verification_prompt_llm["choices"][0]["text"].strip())

{
"question1": "What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?",
"question2": "What was the objective of creating Group Risk Management within AXA?",
"question3": "In which year did AXA return to profitable growth after the economic environment improved?",
"question4": "What was the name of the Swiss firm that AXA acquired in 2006?",
"question5": "What was the purpose of launching the AXA Research Fund?"
}


In [73]:
verification_response=verification_prompt_llm["choices"][0]["text"].strip()
verification_response

'{\n"question1": "What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?",\n"question2": "What was the objective of creating Group Risk Management within AXA?",\n"question3": "In which year did AXA return to profitable growth after the economic environment improved?",\n"question4": "What was the name of the Swiss firm that AXA acquired in 2006?",\n"question5": "What was the purpose of launching the AXA Research Fund?"\n}'

### Step:3 verifying verification response

In [78]:
import json
verification_question_beginning=verification_response.find("{")
json.loads(verification_response[verification_question_beginning:])

{'question1': 'What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?',
 'question2': 'What was the objective of creating Group Risk Management within AXA?',
 'question3': 'In which year did AXA return to profitable growth after the economic environment improved?',
 'question4': 'What was the name of the Swiss firm that AXA acquired in 2006?',
 'question5': 'What was the purpose of launching the AXA Research Fund?'}

In [80]:
verification_factual_questions_dict = json.loads(verification_response[verification_question_beginning:])

In [81]:

verification_factual_questions_dict.values()

dict_values(['What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?', 'What was the objective of creating Group Risk Management within AXA?', 'In which year did AXA return to profitable growth after the economic environment improved?', 'What was the name of the Swiss firm that AXA acquired in 2006?', 'What was the purpose of launching the AXA Research Fund?'])

In [84]:
verification_responses = []
verification_factual_questions_dict.values()

dict_values(['What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?', 'What was the objective of creating Group Risk Management within AXA?', 'In which year did AXA return to profitable growth after the economic environment improved?', 'What was the name of the Swiss firm that AXA acquired in 2006?', 'What was the purpose of launching the AXA Research Fund?'])

In [86]:
verification_responses_template = """Answer the following question correctly based on the context given below:
Context:
{context}
===

Question: {verification_question}"""


In [90]:
for verification_factual_questions in verification_factual_questions_dict.values():
  verification_prompt_question= llama_prompt_template.format(system_message=system_message,
                                                             user_message=verification_prompt.format(
                                                               context= AXA_2000_context,
                                                               verification_question=verification_factual_questions
                                                             ))
  verification_prompt_response=lccp_llm(prompt=verification_prompt_question,temperature=0.8,max_tokens=1500,echo=False,min_p=0.02)
  verification_responses.append(verification_prompt_response["choices"][0]["text"].strip())

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      84.89 ms /   135 runs   (    0.63 ms per token,  1590.39 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   58749.89 ms /   135 runs   (  435.18 ms per token,     2.30 tokens per second)
llama_print_timings:       total time =   58984.93 ms /   135 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      68.59 ms /   113 runs   (    0.61 ms per token,  1647.57 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   48890.26 ms /   113 runs   (  432.66 ms per token,     2.31 tokens per second)
llama_print_timings:       to

In [91]:
for q,a in zip(verification_factual_questions_dict.values(),verification_responses):
  print(q)
  print(a)

What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?
{
"question1": "Can you tell me more about the economic environment that led to AXA's decision to sell off Donaldson, Lufkin & Jenrette (DLJ) in 2001?"
"question2": "What was the main objective of creating Group Risk Management within AXA in 2006?"
"question3": "In what year did AXA return to profitable growth after the subprime financial crisis broke out in 2007?"
"question4": "Can you elaborate on the AXA Research Fund and its focus areas?"
}
What was the objective of creating Group Risk Management within AXA?
{
"question1": "What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?",
"question2": "What was the objective of the new risk management department created by AXA in 2006?",
"question3": "In which year did AXA launch the AXA Research Fund?",
"question4": "What was the name of the Swiss firm that AXA acquired in 2006?"
}
In 

In [92]:
verification_factual_question_answer_pairs = ''

for q, a in zip(verification_factual_questions_dict.values(), verification_responses):
    verification_factual_question_answer_pairs += ('\n' + \
    f"Verification Question: {q}" + '\n' + \
    f"Response: {a}"
)

In [93]:
print(verification_factual_question_answer_pairs)


Verification Question: What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?
Response: {
"question1": "Can you tell me more about the economic environment that led to AXA's decision to sell off Donaldson, Lufkin & Jenrette (DLJ) in 2001?"
"question2": "What was the main objective of creating Group Risk Management within AXA in 2006?"
"question3": "In what year did AXA return to profitable growth after the subprime financial crisis broke out in 2007?"
"question4": "Can you elaborate on the AXA Research Fund and its focus areas?"
}
Verification Question: What was the objective of creating Group Risk Management within AXA?
Response: {
"question1": "What was the name of the US investment bank that Henri de Castries chose to sell off when he took office?",
"question2": "What was the objective of the new risk management department created by AXA in 2006?",
"question3": "In which year did AXA launch the AXA Research Fund?",
"question4": "Wh

### Step4 Final Model

In [94]:
final_prompt_template = """
Given the below `Context`, `Original Query` and `Baseline Answer`, analyze the `Verification Questions & Answers` to finally filter the refined answer.
Context:
{context}
===
Original Query: {original_question}
Baseline Answer: {baseline_response}

Verification Questions & Answer Pairs:
{verification_question_answer_pairs}

To reiterate, you should answer the original query accounting for the veracity of the verification question answer pairs and the context.
Only answer the original question and do not present any additional details. Present your final answer in the following format:
Final Refined Answer:<your-final-answer>"""

In [95]:
final_prompt = llama_prompt_template.format(
    system_message=system_message,
    user_message=final_prompt_template.format(
        context=AXA_2000_context,
        original_question=actual_question_1,
        baseline_response=baseline_response,
        verification_question_answer_pairs=verification_factual_question_answer_pairs
    )
)

In [97]:
response = lccp_llm(
    prompt=final_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   90407.77 ms
llama_print_timings:      sample time =      12.48 ms /    20 runs   (    0.62 ms per token,  1602.95 tokens per second)
llama_print_timings: prompt eval time =  275829.10 ms /  1409 tokens (  195.76 ms per token,     5.11 tokens per second)
llama_print_timings:        eval time =    9216.89 ms /    19 runs   (  485.10 ms per token,     2.06 tokens per second)
llama_print_timings:       total time =  285108.63 ms /  1428 tokens


In [98]:
print(response["choices"][0]["text"].strip())

Final Refined Answer: The year of the financial crisis is 2007.
